In [11]:
import ComposeFunction
import ConfigReader
import FuzzyMatch
import StringFormat

In [31]:
import FuzzyMatch as fm
import ComposeFunction as cf
import pandas as pd
import numpy as np

path = "tests/"


# Load data
def load_match_data():
    companies = pd.read_csv(path + "data/fuzzy_match_companies.csv")
    return companies


# Manually specified cleaning pipeline
def company_cleaning_pipeline(dat_in):
    fns = [
        "StringFormat.to_lower",
        "StringFormat.trim_whitespace",
        "StringFormat.strip_non_ascii",
        "StringFormat.strip_punctuation",
        "FuzzyMatch.remove_company_suffixes",
    ]
    out = list()
    if type(dat_in) is str:
        return cf.eval_functions_list(dat_in, fns)
    else:
        for dat in dat_in:
            out.append(cf.eval_functions_list(dat, fns))
    return out

In [27]:
# Load data
companies = load_match_data()
ref_name = company_cleaning_pipeline(companies["reference_name"])
new_name = company_cleaning_pipeline(companies["new_name"])

# Test output
scores = fm.fuzzy_match_pairwise(ref_name, new_name)
print(scores)

[100, 90, 87, 100, 90, 90, 90, 86, 58, 50, 73, 61, 62, 67, 30, 19]


In [28]:
companies["scores"] = scores
pd.options.display.width = 160
print(companies)

                    reference_name                        new_name expected  scores
0          Ballentine Partners LLC        Ballentine Partners, LLC    match     100
1       Huron Capital Partners LLC                   Huron Capital    match      90
2                USI Insurance Svc          Usi Insurance Services    match      87
3        New Heritage Capital, LLC            New Heritage Capital    match     100
4                           Aureus                  Aureus Capital    match      90
5                  Marsh USA, Inc.                       Marsh Inc    match      90
6              Madison Capital LLC         Madison Capital Funding    match      90
7        Adams Street Partners LLC              Partners, Adams St    match      86
8              Promus Bank & Trust              Promus Capital LLC      new      58
9           Huron Consulting Group      Huron Capital Partners LLC      new      50
10  Comprehensive Capital Partners  Cohesive Capital Planning L.P.      new 

In [29]:
companies["scores"] >= 80 

0      True
1      True
2      True
3      True
4      True
5      True
6      True
7      True
8     False
9     False
10    False
11    False
12    False
13    False
14    False
15    False
Name: scores, dtype: bool

In [35]:
companies["result"] = np.where(companies["scores"] >= 80, "match", "new")
companies["expected"].to_list() == companies["result"].to_list()

False